## Exploratory analysis of the Buenos Aires GTFS for buses
---

In [134]:
import polars as pl

In [135]:
import glob
import os

In [40]:
calendar_dates = pl.read_csv(os.path.join('source','calendar_dates.txt'))
trips = pl.read_csv(os.path.join('source','trips.txt'))
routes = pl.read_csv(os.path.join('source','routes.txt'))
shapes = pl.read_csv(os.path.join('source','shapes.txt'))
stop_times = pl.read_csv(os.path.join('source','stop_times.txt'))
agency = pl.read_csv(os.path.join('source','agency.txt'))

**Searching for null values in each DataFrame**

In [41]:
path = os.path.join('source', '*.txt')
files = [os.path.basename(f)[0:-4] for f in glob.glob(path)] # Delete extension from the file names through splitting

In [42]:
for table in files:
    nulls = 0
    columns = globals()[table].get_columns()
    for col in columns:
        nulls += col.is_null().sum()
    print(f'{nulls} null values in the table {table.upper()}')

0 null values in the table AGENCY
0 null values in the table CALENDAR_DATES
0 null values in the table ROUTES
0 null values in the table SHAPES
0 null values in the table STOP_TIMES
0 null values in the table TRIPS


---
### Structure of `calendar_dates`

In [81]:
calendar_dates

service_id,date,exception_type,year,month,day
i64,str,i64,i32,i8,i8
1,"""20190928""",1,2019,9,28
2,"""20190929""",1,2019,9,29
3,"""20190930""",1,2019,9,30
3,"""20191001""",1,2019,10,1
3,"""20191002""",1,2019,10,2
…,…,…,…,…,…
3,"""20191227""",1,2019,12,27
1,"""20191228""",1,2019,12,28
2,"""20191229""",1,2019,12,29


There are **4 types** of services attached to each trip, with service 3 being the one with most days associated 

In [44]:
calendar_dates.group_by('service_id').len('total_days').sort('total_days', descending=True)

service_id,total_days
i64,u32
3,63
1,15
2,13
4,4


In [45]:
# Cast 'date' column to str
calendar_dates = calendar_dates.with_columns(
    pl.col('date').cast(pl.Utf8)
)

In [46]:
# Partition the 'date' column into year, month and day columns
calendar_dates = calendar_dates.with_columns([
    pl.col('date').str.strptime(pl.Date, '%Y%m%d').dt.year().alias('year'),
    pl.col('date').str.strptime(pl.Date, '%Y%m%d').dt.month().alias('month'),
    pl.col('date').str.strptime(pl.Date, '%Y%m%d').dt.day().alias('day')
])

Comparing the list of days for each month in each service in the `services_calendar` DataFrame with the Argentinian national calendar, the following attributes of each service type can be inferred:

* Service 1 is for saturdays and holiday vespers
* Service 2 is for sundays
* Service 3 is for weekdays
* Service 4 is for holidays

In [47]:
services_calendar = (
    calendar_dates
    .group_by(['service_id', 'year', 'month'])
    .agg(pl.col('day').alias('days'))
    .sort(['service_id', 'year', 'month'])
)

Let's take for example service 4

In [48]:
# Here, each day for each month is a national holiday in Argentina for the given year
services_calendar.filter(pl.col('service_id') == 4)

service_id,year,month,days
i64,i32,i8,list[i8]
4,2019,10,[12]
4,2019,11,[18]
4,2019,12,"[8, 25]"


---
### Structure of `trips`

In [80]:
trips

route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,exceptional
i64,i64,str,str,str,i64,str,i64,i64
100,1,"""1-1""","""a Pque. Avellaneda""","""100SI0001""",0,"""100SI0001""",1,0
100,1,"""2-1""","""a Pque. Avellaneda""","""100SI0002""",0,"""100SI0002""",1,0
100,1,"""3-1""","""a Pque. Avellaneda""","""100SI0003""",0,"""100SI0003""",1,0
100,1,"""4-1""","""a Pque. Avellaneda""","""100SI0004""",0,"""100SI0004""",1,0
100,1,"""5-1""","""a Pque. Avellaneda""","""100SI0005""",0,"""100SI0005""",1,0
…,…,…,…,…,…,…,…,…
99,4,"""436403-1""","""a Pza. Miserere""","""99FI1038""",1,"""99FI1038""",2066,0
99,4,"""436404-1""","""a Pza. Miserere""","""99FI1039""",1,"""99FI1039""",2066,0
99,4,"""436405-1""","""a Pza. Miserere""","""99FI1040""",1,"""99FI1040""",2066,0


The number of trips per day for each route can be analyzed using COUNT to later be joined with the `routes` file with each bus information, taking into account inbound and outbound trips as seen in the next example using route 100

In [50]:
# Note that the number of trips for the inbound and outbound directions is unequal
trips.group_by('route_id','direction_id').len('total_trips').filter(pl.col('route_id') == 100)

route_id,direction_id,total_trips
i64,i64,u32
100,1,364
100,0,366


In [51]:
# Instead, the mean can be taken from both
mean_trips_per_route = (
    trips
    .group_by('route_id', 'direction_id')
    .len('total_trips')
    .group_by('route_id')
    .agg(pl.col('total_trips').mean().alias('route_trips'))
    .with_columns(
        pl.col('route_trips').round().cast(pl.Int32)
    )
)

In [52]:
mean_trips_per_route.filter(pl.col('route_id') == 100)

route_id,route_trips
i64,i32
100,365


Given the difference in attributes between directions and even services (as seen in `stop_times`), the following cells will be useful to measure attributes of routes and bus lines taking into account only one representative trip for each route in both directions or in both directions within each service

**TRIP-CASE-1**

In [53]:
service_trip_of_routes = (
    trips
    .group_by('route_id', 'service_id', 'direction_id', 'shape_id')
    .agg(pl.col('trip_id').first())
)

**TRIP-CASE-2**

In [54]:
trip_of_routes = (
    service_trip_of_routes
    .group_by('route_id', 'direction_id', 'shape_id')
    .agg(pl.col('trip_id').first())
)

_TRIP-CASE-1 would already satisfy the requirements for both cases when computing means between values dependent on services and directions or solely dependent on directions, but the latter scenario would imply redundant operations which can slower down the overall performance, so TRIP-CASE-2 is created from TRIP-CASE-1 to minimize compute times since grouping by service, in those cases, is not necessary_

---
### Structure of `routes`

In [82]:
routes

route_id,agency_id,route_short_name,route_long_name,route_desc,route_type
i64,i64,str,str,str,i64
4167,110,"""505R3""","""JMALBR505""","""Ramal 3 - San Francisco Solano…",3
4168,110,"""505R4""","""JMALBR505""","""Ramal 4 - San Francisco Solano…",3
4169,110,"""505R5""","""JMALBR505""","""Ramal 5 - San Francisco Solano…",3
4170,110,"""505R6""","""JMALBR505""","""Ramal 6 - San Francisco Solano…",3
4171,110,"""506R1""","""JMALBR506""","""Ramal 1 - Est. Glew - Tapin y …",3
…,…,…,…,…,…
1478,115,"""463D""","""JPBSAS463""","""x Ruben Dario""",3
1482,115,"""464D""","""JPBSAS464""","""Ramal D - Felix Frias""",3
1479,115,"""464A""","""JPBSAS464""","""Ramal x Cinturon Ecologico""",3


To make aggregations over the bus lines without taking into account each route branch ("ramal"), the column route_short_name should be used to create a derived column with only the bus line name. As seen from the next cell, outlier values are found where bus names are completely alphabetic and not numeric as the mayority of cases

In [56]:
routes.filter(pl.col('route_short_name').str.contains(r'^([A-Z]+)')).select('route_short_name').head()

route_short_name
str
"""AZUL1"""
"""AZUL2"""
"""ROJA"""
"""VERDE"""
"""NORTE12"""


In [57]:
# It is seen that the alphabetic values are scarce along the dataset
routes.with_columns(
    pl.col('route_short_name')
    .str.contains(r'^([A-Z]+)')
    .alias('letters_first')
).group_by('letters_first').len('route_id')

letters_first,route_id
bool,u32
false,1034
true,18


Conditional statements can be used both in SQL and in `polars` to separate both cases as following

In [58]:
routes_with_buses = (
    routes.
    with_columns(
        pl.when(pl.col('route_short_name').str.contains(r'^\d+'))
        .then(pl.col('route_short_name').str.extract(r'^(\d+)', 1))  # Digits before letters
        .otherwise(pl.col('route_short_name').str.extract(r'^([A-Z]+)', 1)) # Letters at the beginning
        .alias('bus_line')
    )
)

Then, for example, the count of route branches for each bus line can be calculated and sorted

In [59]:
routes_with_buses.group_by('bus_line').len('branches').sort('branches', descending=True).head()

bus_line,branches
str,u32
"""96""",35
"""60""",20
"""620""",19
"""57""",19
"""129""",18


---
### Structure of `shapes`

In [83]:
shapes

shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
i64,f64,f64,i64,f64
418,-34.601438,-58.454088,53,3628.46
418,-34.602127,-58.455055,54,3745.43
418,-34.602792,-58.455998,55,3859.09
418,-34.603757,-58.457437,56,4028.91
418,-34.604258,-58.458182,57,4117.0
…,…,…,…,…
2017,-34.545957,-58.489405,176,12126.9
2017,-34.545477,-58.488467,177,12228.0
2017,-34.545328,-58.488163,178,12260.2


The overall distance completed by each route can be obtained by getting the largest shape_dist_traveled value of the shape_id associated with each representative trip of a route by direction through trip_of_routes (TRIP-CASE-2), as seen in the following example

In [61]:
total_dist_per_trip = (
    shapes
    .group_by('shape_id')
    .agg(pl.col('shape_dist_traveled').max().alias('trip_distance'))
    .join(trip_of_routes, 'shape_id') # It has to be an INNER join
)

In [62]:
total_dist_per_trip.sort('route_id').head()

shape_id,trip_distance,route_id,direction_id,trip_id
i64,f64,i64,i64,str
211,15907.0,12,1,"""123551-1"""
210,16437.0,12,0,"""12074-1"""
1702,19215.0,44,1,"""419921-1"""
1701,21325.0,44,0,"""419831-1"""
1715,26395.0,45,0,"""324005-1"""


Then, the average is computed from both directions, grouped by route_id, and can be ultimately joined with the `routes` table

In [63]:
total_dist_per_trip. \
    group_by('route_id'). \
    agg(pl.col('trip_distance').mean().round().alias('route_avg_distance')). \
    head()

route_id,route_avg_distance
i64,f64
2519,29490.0
3695,19941.0
1590,12594.0
1206,38329.0
2507,23258.0


---
### Structure of `stop_times`

In [84]:
stop_times

trip_id,arrival_time,departure_time,stop_id,stop_sequence,timepoint,shape_dist_traveled
str,str,str,i64,i64,i64,i64
"""1-1""","""00:26:00""","""00:26:00""",205696,1,1,0
"""1-1""","""00:28:00""","""00:28:00""",204229,2,0,514
"""1-1""","""00:29:08""","""00:29:08""",204191,3,0,803
"""1-1""","""00:31:06""","""00:31:06""",205517,4,0,1306
"""1-1""","""00:34:12""","""00:34:12""",205528,5,0,2093
…,…,…,…,…,…,…
"""436407-1""","""28:56:40""","""28:56:40""",204910,24,0,86746
"""436407-1""","""28:58:24""","""28:58:24""",201247,25,0,87960
"""436407-1""","""29:00:36""","""29:00:36""",203260,26,0,89480


It can be seen from the second cell that the number of stops is equal for some trips. This can be due to coincidence or due to the trips belonging to one specific route

In [143]:
stops_per_trip = stop_times.group_by('trip_id').len('trip_stops')

In [144]:
stops_per_trip.sort('trip_stops', descending=True).head()

trip_id,trip_stops
str,u32
"""42633-1""",222
"""388197-1""",222
"""149288-1""",222
"""42647-1""",222
"""388165-1""",222


To segregate both cases, one individual trip is taken for each direction_id of each route_id through trip_of_routes (TRIP-CASE-2), also enabling computing the overall number of stops per route

In [75]:
trip_of_routes.join(stops_per_trip, 'trip_id').sort('route_id').head()

route_id,direction_id,shape_id,trip_id,trip_stops
i64,i64,i64,str,u32
12,1,211,"""123551-1""",95
12,0,210,"""12074-1""",103
44,1,1702,"""419921-1""",62
44,0,1701,"""419831-1""",61
45,1,1716,"""190223-1""",64


Then, the average is computed from both directions, grouped by route_id, and can be ultimately joined with the `routes` table

In [67]:
trip_of_routes. \
    join(stops_per_trip, 'trip_id'). \
    group_by('route_id'). \
    agg(pl.col('trip_stops').mean().round().alias('route_avg_stops')). \
    head()

route_id,route_avg_stops
i64,f64
2906,14.0
143,70.0
2909,68.0
1736,92.0
1727,40.0


On the other hand, the times used in arrival_time and departure_time exceed the 24 hour system, using hour measures such as 26, 32, etc. To handle this issue properly when calculating total trip times, the minimum and the maximum times should be lexicographically taken from this column using the original format and the operation must be done:
* Subtracting 24 hours to both times if **both** have hour digits higher or equal to 24 or...
* Subtracting 12 hours to both times if **only the end time** has hour digits higher or equal to 24

The next functions are built so that both cases can be treated with them either in individual cases (1) or within the whole dataset (2) using the library's built-in function `map_elements`, which for this demo satisfies the requirements but is not recommended for big DataFrames (see [polars.Expr.map_elements](https://docs.pola.rs/api/python/dev/reference/expressions/api/polars.Expr.map_elements.html))

**FUNCTION 1**

In [ ]:
def format_gtfs_time(time: str, offset: int = 24) -> str:
    """
    Format a GTFS time with hour values ranging from '00' up to '47' 
    into a standard value within the 24-hour system.

    Args:
        time: The original GTFS time value as a 'HH:MM:SS' string.
        offset: The offset by which the hour digits will be shifted 
                counterclockwise. The default value is by 24 hours, but
                smaller units can be passed only for exceptional scenarios
                where times with both normal or abnormal hour measures
                should be shifted equally.

    Returns:
        str: A value ranging from 00:00:00 to 23:59:59 included.
    """
    if offset < 25:
        hour = int(time[:2])
        rest = time[2:]
        
        if hour >= 24 or offset != 24:
            hour -= offset
        new_time = '0' + str(hour) + rest
        return new_time[-8:]
    else:
        raise Exception('Offset should be less or equal than 24 hours')

**FUNCTION 2**

In [117]:
from datetime import datetime

In [160]:
def gtfs_time_difference(start_time: str, end_time: str) -> int:
    """
    Get the time difference between two GTFS times in minutes, 
    previously formatting both within the 24-hour system given
    two scenarios:
    
    - Both have hour digits higher or equal to 24
    - Only the end time has hour digits higher or equal to 24

    Args:
        start_time: The GTFS start time value as a 'HH:MM:SS' string.
        end_time: The GTFS end time value as a 'HH:MM:SS' string.

    Returns:
        int: The time difference in minutes.
    """
    start_hour = int(start_time[:2])
    end_hour = int(end_time[:2])
    
    if start_hour > 23 and end_hour > 23:
        # Offset is 24 hours
        start_time = datetime.strptime(format_gtfs_time(start_time), '%H:%M:%S')
        end_time = datetime.strptime(format_gtfs_time(end_time), '%H:%M:%S')
    elif start_hour < 24 and end_hour > 23:
        # Offset is 12 hours
        start_time = datetime.strptime(format_gtfs_time(start_time, offset=12), '%H:%M:%S')
        end_time = datetime.strptime(format_gtfs_time(end_time, offset=12), '%H:%M:%S')
    else:
        # No offset for already formatted times
        start_time = datetime.strptime(start_time, '%H:%M:%S')
        end_time = datetime.strptime(end_time, '%H:%M:%S')
    
    time_difference = (end_time - start_time).total_seconds() / 60
    return int(time_difference)

Let's take the minimum and maximum times for each trip through string comparisons

In [171]:
trip_times = (
    stop_times
    .group_by('trip_id')
    .agg(
        pl.col('arrival_time').min().alias('min_time'),
        pl.col('arrival_time').max().alias('max_time')
    )   
)

**Example with both times starting with hour measures higher or equal than 24**

In [172]:
# ORIGINAL
trip_times.filter(pl.col('trip_id') == '143421-1')

trip_id,min_time,max_time
str,str,str
"""143421-1""","""25:30:00""","""26:33:56"""


In [173]:
# FORMATTED
trip_times. \
    filter(pl.col('trip_id') == '143421-1'). \
    with_columns([
        pl.col('min_time').map_elements(format_gtfs_time, return_dtype=pl.String).alias('min_time'),
        pl.col('max_time').map_elements(format_gtfs_time, return_dtype=pl.String).alias('max_time')
    ])
    

trip_id,min_time,max_time
str,str,str
"""143421-1""","""01:30:00""","""02:33:56"""


**Example with only the end time starting with an hour measure higher or equal than 24**

_Note that, unlike the previous case, the new times have a variant offset which does not preserve the identity of the original values, but are still useful to get accurate time differences_

In [148]:
# ORIGINAL
trip_times.filter(pl.col('trip_id') == '436399-1')

trip_id,min_time,max_time
str,str,str
"""436399-1""","""22:20:00""","""24:33:00"""


In [174]:
# FORMATTED
trip_times. \
    filter(pl.col('trip_id') == '436399-1'). \
    with_columns(
        pl.col('min_time').map_elements(lambda x: format_gtfs_time(x, 12), return_dtype=pl.String).alias('min_time'),
        pl.col('max_time').map_elements(lambda x: format_gtfs_time(x, 12), return_dtype=pl.String).alias('max_time')
    )

trip_id,min_time,max_time
str,str,str
"""436399-1""","""10:20:00""","""12:33:00"""


Although individual trips grouped by route_id and direction_id were selected through TRIP-CASE-2 for previous average measurements, stop times and therefore total trip times can also change according to the service type that the trips belong to

> This property of stop times was clarified in later testing where final average route times, taken with the TRIP-CASE-2 approach, were mostly unequal to average route times computed with arbitrary trips

In [175]:
trip_times_diff = (
    trip_times
    .with_columns(
        pl.struct(['min_time', 'max_time']) # Get both columns to compute the difference
        .map_elements(lambda x: gtfs_time_difference(x['min_time'], x['max_time']), return_dtype=pl.Int16)
        .alias('total_minutes')
    )
    .select('trip_id', 'total_minutes')
)

In [186]:
service_trip_of_routes. \
    join(trip_times_diff, 'trip_id'). \
    filter(pl.col('route_id') == 100). \
    sort('service_id')

route_id,service_id,direction_id,shape_id,trip_id,total_minutes
i64,i64,i64,i64,str,i16
100,1,0,1,"""1-1""",64
100,1,1,2,"""89-1""",64
100,2,1,2,"""112957-1""",64
100,2,0,1,"""112890-1""",64
100,3,0,1,"""208858-1""",68
100,3,1,2,"""209002-1""",68
100,4,0,1,"""354848-1""",64
100,4,1,2,"""354915-1""",64


Then, the average is computed from both directions, grouped by route_id, and can be ultimately joined with the `routes` table

In [191]:
service_trip_of_routes. \
    join(trip_times_diff, 'trip_id'). \
    group_by('route_id'). \
    agg(pl.col('total_minutes').mean().round().alias('route_avg_time')). \
    head()

route_id,route_avg_time
i64,f64
1581,46.0
277,90.0
3028,40.0
283,141.0
798,38.0
